In [50]:
import mlflow
import pandas as pd
import matplotlib.pyplot as plt

In [51]:
algorithms = ['PL', 'STC', 'CC', 'HLNC', 'OBNC', 'BE']

In [52]:
runs = {}
for dataset in ['adult', 'german', 'compas', 'ricci', 'diabetes']:
    for attr in ['sex_Male', 'race_White', 'sex', 'race_Caucasian', 'Race_W']:
        for alg in algorithms:
            run = mlflow.search_runs(experiment_names=[f'{dataset}_{attr}_{alg}'], order_by=['start_time DESC'])[:4]
            if len(run) > 0:
                runs[f'{dataset}_{attr}_{alg}'] = run

In [53]:
metrics = [
    'accuracy',
    'equal_opportunity_difference', 
    'predictive_equality_difference',
    'demographic_parity_difference',
    'equalized_odds_difference']

In [54]:
results = {}

for alg in algorithms:
    results_df = pd.DataFrame(columns=metrics)

    for exp in ['adult_sex_Male', 'adult_race_White', 'german_sex_Male', 'compas_sex', 'compas_race_Caucasian', 'ricci_Race_W', 'diabetes_race_Caucasian']:
        if f'{exp}_{alg}' not in runs:
            print(f'{exp}_{alg} must be repeated')
            continue
        run = runs[f'{exp}_{alg}']
        noisy = run.loc[(run['tags.train_set'] == 'noisy') & (run['tags.test_set'] == 'noisy')]
        corrected = run.loc[(run['tags.train_set'] == 'corrected') & (run['tags.test_set'] == 'noisy')]

        results_df.loc[exp] = [
            (corrected[f'metrics.{metric}'].values[0] - noisy[f'metrics.{metric}'].values[0])/noisy[f'metrics.{metric}'].values[0] 
            if noisy[f'metrics.{metric}'].values[0] != 0
            else 0 
            for metric in metrics]
        
    results[alg] = results_df

compas_sex_STC must be repeated
compas_race_Caucasian_STC must be repeated
ricci_Race_W_CC must be repeated
compas_race_Caucasian_HLNC must be repeated
ricci_Race_W_HLNC must be repeated
german_sex_Male_BE must be repeated


In [69]:
for metric in metrics:
    results_df = pd.DataFrame(columns=['mean', 'std', 'min', 'max'])
    for alg in algorithms:
        results_df.loc[alg] = results[alg][metric].describe()[['mean', 'std', 'min', 'max']].values
    results_df.to_csv(f'results/aggregated_results/{metric}.csv')

In [70]:
results_df = pd.DataFrame(columns=metrics)
for alg in algorithms:
    results_df.loc[alg] = [results[alg][metric].describe()['mean'] for metric in metrics]
results_df.to_csv(f'results/aggregated_results/mean_results.csv')

In [77]:
results['HLNC']['predictive_equality_difference'].describe()[['mean', 'std', 'min', 'max']]

mean   -0.533351
std     0.488329
min    -0.926624
max     0.000000
Name: predictive_equality_difference, dtype: float64